In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import pandas as pd
import numpy as np
from joblib import load,dump

In [0]:
farm_data_master=pd.read_csv('/content/drive/My Drive/Phd/new_farm_data.csv')
train_data_master=pd.read_csv('/content/drive/My Drive/Phd/train_data.csv')
train_weather_master=pd.read_csv('/content/drive/My Drive/Phd/train_weather.csv')

In [0]:
# We are creating an temperory dataframe 366*24 rows of timestamps across 16 locations and then we merge it with train_weather_master
temp_location_list=[]
temp_timestamp_list=[]
for i in train_weather_master.deidentified_location.unique():
    for j in pd.date_range('2016-01-01 00:00:00', periods=8784, freq='H'):
        temp_timestamp_list.append(j)
        temp_location_list.append(i)

new_train_weather_master=pd.DataFrame(columns=['timestamp', 'deidentified_location'])
new_train_weather_master.timestamp=pd.Series(temp_timestamp_list)
new_train_weather_master.deidentified_location=pd.Series(temp_location_list)

In [0]:
train_weather_master.timestamp=train_weather_master.timestamp.astype('datetime64[ns]')

In [0]:
weather_merge=pd.merge(train_weather_master, new_train_weather_master, on=['timestamp', 'deidentified_location'], how='outer')
weather_merge=weather_merge.sort_values(by=['deidentified_location','timestamp']).reset_index(drop=True)

In [0]:
for location in weather_merge.deidentified_location.unique():
  weather_merge[weather_merge.deidentified_location==location].to_csv(("/content/drive/My Drive/Phd/weather_data_location_wise/"+location+".csv"),index=False)


In [94]:
for location in weather_merge.deidentified_location.unique():
  temp_dataframe=pd.read_csv(('/content/drive/My Drive/Phd/weather_data_location_wise/'+location+'.csv'))
  for column in ['temp_obs', 'cloudiness','wind_direction', 'dew_temp', 'pressure_sea_level', 'precipitation','wind_speed']:
        for index,value in zip(temp_dataframe[column].isna().value_counts().index,temp_dataframe[column].isna().value_counts()):
          if (index and value==365*24):
            continue
          else:
            data1=temp_dataframe[column]
            mask = np.isnan(data1)
            data1[mask] = np.interp(np.flatnonzero(mask), np.flatnonzero(~mask), data1[~mask])
            temp_dataframe[column]=data1
  temp_dataframe.to_csv(('/content/drive/My Drive/Phd/weather_data_imputations/'+location+'.csv'),index=False)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


In [0]:
weather_imputation_full=pd.DataFrame()
for location in weather_merge.deidentified_location.unique():
  temp_dataframe=pd.read_csv(('/content/drive/My Drive/Phd/weather_data_imputations/'+location+'.csv'))
  if location==weather_merge.deidentified_location.unique()[0]:
    weather_imputation_full=temp_dataframe
  else:
    weather_imputation_full=weather_imputation_full.append(temp_dataframe,ignore_index=True)

In [0]:
for column in ['temp_obs', 'cloudiness','wind_direction', 'dew_temp', 'pressure_sea_level', 'precipitation','wind_speed']:
  weather_imputation_full[column].fillna(weather_imputation_full[column].mean(),inplace=True)

In [97]:
dump(weather_imputation_full,'/content/drive/My Drive/Phd/weather_data_Testfull.joblib')

['/content/drive/My Drive/Phd/weather_data_Testfull.joblib']

In [0]:
final_train=train_data_master.groupby(by=['date', 'farm_id', 'ingredient_type'],as_index=False).sum()

In [0]:
weather_imputation_full.timestamp=weather_imputation_full.timestamp.astype('datetime64[ns]')
final_train.rename(columns={'date':'timestamp'},inplace=True)
final_train.timestamp=final_train.timestamp.astype('datetime64[ns]')

In [24]:
final_merge=pd.merge(pd.merge(final_train, farm_data_master, on=['farm_id'], how='left'), weather_imputation_full, on=['timestamp', 'deidentified_location'], how='left')
dump(final_merge,'/content/drive/My Drive/Phd/train_data_raw.joblib')

['/content/drive/My Drive/Phd/train_data_raw.joblib']

In [27]:
np.percentile(final_merge['yield'],97)

2516.19

In [0]:
final_merge=final_merge[final_merge['yield']<=2516.19]

In [90]:
final_merge.drop(['operations_commencing_year', 'num_processing_plants'],axis=1,inplace=True)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [91]:
dump(final_merge,'/content/drive/My Drive/Phd/train_data_removied_outliers_raw.joblib')

['/content/drive/My Drive/Phd/train_data_removied_outliers_raw.joblib']

In [42]:
final_merge.groupby(by=['farm_id','ingredient_type'],as_index=False).sum()[['farm_id','ingredient_type','yield']]

,farm_id,ingredient_type,yield
0,fid_100053,ing_w,4.722934e+05
1,fid_100107,ing_w,4.773916e+06
2,fid_100107,ing_x,3.377998e+06
3,fid_100107,ing_y,2.678898e+06
4,fid_100135,ing_w,2.196656e+05
...,...,...,...
2358,fid_99921,ing_w,1.285013e+06
2359,fid_99921,ing_x,2.144410e+06
2360,fid_99942,ing_w,1.313502e+06
2361,fid_99942,ing_x,2.571440e+06


In [38]:
final_merge.groupby(by=['timestamp','ingredient_type'],as_index=False).sum()[['timestamp','ingredient_type','yield']]

,timestamp,ingredient_type,yield
0,2016-01-01 00:00:00,ing_w,135023.2208
1,2016-01-01 00:00:00,ing_x,40713.7369
2,2016-01-01 00:00:00,ing_y,136737.5590
3,2016-01-01 00:00:00,ing_z,42975.7381
4,2016-01-01 01:00:00,ing_w,154823.1326
...,...,...,...
35131,2016-12-31 22:00:00,ing_z,39119.0558
35132,2016-12-31 23:00:00,ing_w,186261.5612
35133,2016-12-31 23:00:00,ing_x,72540.8947
35134,2016-12-31 23:00:00,ing_y,166238.7432


In [39]:
366*24*4

35136

In [0]:
file_paths_master=pd.read_csv('/content/drive/My Drive/Phd/file_paths.csv')

In [0]:
file_paths_master.drop('file_paths',inplace=True,axis=1)

In [0]:
fram_id_traget_encoding=pd.merge(file_paths_master,final_merge.groupby(by=['farm_id','ingredient_type'],as_index=False).sum()[['farm_id','ingredient_type','yield']],on=['farm_id','ingredient_type'],how='left').fillna(0)

In [22]:
dump(farm_id_traget_encoding,'/content/drive/My Drive/Phd/farm_id_target_encoding.joblib')

['/content/drive/My Drive/Phd/farm_id_target_encoding.joblib']

In [51]:
fram_id_traget_encoding

,farm_id,ingredient_type,yeild_nan_count,yield
0,fid_110884,ing_w,0,1.286461e+06
1,fid_110884,ing_z,8784,0.000000e+00
2,fid_110884,ing_x,8784,0.000000e+00
3,fid_110884,ing_y,8784,0.000000e+00
4,fid_90053,ing_w,0,6.576176e+05
...,...,...,...,...
5731,fid_60652,ing_y,6127,7.696698e+05
5732,fid_19864,ing_w,8305,2.422705e+04
5733,fid_19864,ing_z,8784,0.000000e+00
5734,fid_19864,ing_x,8784,0.000000e+00


In [56]:
farming_company_weights=final_merge.groupby(by=['farming_company','ingredient_type'],as_index=False).sum()[['farming_company','ingredient_type','yield']].rename(columns={'yield':'farming_company_weights'})
dump(farming_company_weights,'/content/drive/My Drive/Phd/farming_company_weights.joblib')

['/content/drive/My Drive/Phd/farming_company_weights.joblib']

In [53]:
final_merge.columns

Index(['timestamp', 'farm_id', 'ingredient_type', 'yield',
       'operations_commencing_year', 'num_processing_plants', 'farm_area',
       'farming_company', 'deidentified_location', 'temp_obs', 'cloudiness',
       'wind_direction', 'dew_temp', 'pressure_sea_level', 'precipitation',
       'wind_speed'],
      dtype='object')

In [21]:
deidentified_location_weights=final_merge.groupby(by=['deidentified_location','ingredient_type'],as_index=False).sum()[['deidentified_location','ingredient_type','yield']].rename(columns={'yield':'deidentified_location_weights'})
dump(deidentified_location_weights,'/content/drive/My Drive/Phd/deidentified_location_weights.joblib')

['/content/drive/My Drive/Phd/deidentified_location_weights.joblib']

In [0]:
global_time_series=final_merge.groupby(by=['timestamp','ingredient_type'],as_index=False).sum()[['timestamp','ingredient_type','yield']]

In [0]:
for ingredient_type in ['ing_w','ing_x','ing_y','ing_z']:
    global_time_series[global_time_series['ingredient_type']==ingredient_type].to_csv(('/content/drive/My Drive/Phd/global_time_series/'+ingredient_type+'.csv'),index=False)

In [0]:
for ingredient_type in ['ing_w','ing_x','ing_y','ing_z']:
    temp=pd.read_csv(('/content/drive/My Drive/Phd/global_time_series/'+ingredient_type+'.csv'))
    temp['month']=pd.DatetimeIndex(temp['timestamp']).month
    temp['day']=pd.DatetimeIndex(temp['timestamp']).day
    temp['hour']=pd.DatetimeIndex(temp['timestamp']).hour
    temp['weekday']=pd.DatetimeIndex(temp['timestamp']).weekday
    temp['quarter']=pd.DatetimeIndex(temp['timestamp']).quarter
    temp['weekofyear']=pd.DatetimeIndex(temp['timestamp']).weekofyear
    temp['month']=pd.merge(temp['month'],temp.groupby('month',as_index=False).sum()[['month','yield']],on=['month'],how='left')['yield']
    temp['day']=pd.merge(temp['day'],temp.groupby('day',as_index=False).sum()[['day','yield']],on=['day'],how='left')['yield']
    temp['hour']=pd.merge(temp['hour'],temp.groupby('hour',as_index=False).sum()[['hour','yield']],on=['hour'],how='left')['yield']
    temp['weekday']=pd.merge(temp['weekday'],temp.groupby('weekday',as_index=False).sum()[['weekday','yield']],on=['weekday'],how='left')['yield']
    temp['quarter']=pd.merge(temp['quarter'],temp.groupby('quarter',as_index=False).sum()[['quarter','yield']],on=['quarter'],how='left')['yield']
    temp['weekofyear']=pd.merge(temp['weekofyear'],temp.groupby('weekofyear',as_index=False).sum()[['weekofyear','yield']],on=['weekofyear'],how='left')['yield']
    temp.rename(columns={'yield':'globaltime_yield_ingredient_weights'},inplace=True)
    temp.rename(columns={'month':'globaltime_month12_ingredient_weights'},inplace=True)
    temp.rename(columns={'day':'globaltime_day31_ingredient_weights'},inplace=True)
    temp.rename(columns={'hour':'globaltime_hour24_ingredient_weights'},inplace=True)
    temp.rename(columns={'weekday':'globaltime_weekday7_ingredient_weights'},inplace=True)
    temp.rename(columns={'quarter':'globaltime_quarter4_ingredient_weights'},inplace=True)
    temp.rename(columns={'weekofyear':'globaltime_weekofyear53_ingredient_weights'},inplace=True)
    temp.to_csv(('/content/drive/My Drive/Phd/global_time_series/'+ingredient_type+'.csv'),index=False)



In [0]:
global_time_series_full=pd.DataFrame()
for ingredient_type in ['ing_w','ing_x','ing_y','ing_z']:
  temp_dataframe=pd.read_csv(('/content/drive/My Drive/Phd/global_time_series/'+ingredient_type+'.csv'))
  if ingredient_type=='ing_w':
    global_time_series_full=temp_dataframe
  else:
    global_time_series_full=global_time_series_full.append(temp_dataframe,ignore_index=True)

In [88]:
dump(global_time_series_full,'/content/drive/My Drive/Phd/global_time_series_full.joblib')

['/content/drive/My Drive/Phd/global_time_series_full.joblib']

In [92]:
final_merge

,timestamp,farm_id,ingredient_type,yield,farm_area,farming_company,deidentified_location,temp_obs,cloudiness,wind_direction,dew_temp,pressure_sea_level,precipitation,wind_speed
0,2016-01-01 00:00:00,fid_100053,ing_w,37.0400,2043.866000,Del Monte Foods,location 5290,10.0,8.0,350.0,2.2,1021.1,0.0,4.1
1,2016-01-01 00:00:00,fid_100107,ing_w,291.8000,35416.574563,Del Monte Foods,location 2532,-8.3,8.0,250.0,-12.2,1023.3,-1.0,4.1
2,2016-01-01 00:00:00,fid_100107,ing_x,25.3232,35416.574563,Del Monte Foods,location 2532,-8.3,8.0,250.0,-12.2,1023.3,-1.0,4.1
4,2016-01-01 00:00:00,fid_100135,ing_w,15.6400,1432.564260,Dole Food Company,location 5290,10.0,8.0,350.0,2.2,1021.1,0.0,4.1
5,2016-01-01 00:00:00,fid_100249,ing_w,19.4900,941.571905,Wayne Farms,location 5489,15.6,6.0,270.0,-5.6,1015.3,0.0,3.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20080527,2016-12-31 23:00:00,fid_99921,ing_w,142.3700,15232.282977,Wayne Farms,location 5489,16.7,4.0,250.0,10.0,1011.0,0.0,2.6
20080528,2016-12-31 23:00:00,fid_99921,ing_x,102.3400,15232.282977,Wayne Farms,location 5489,16.7,4.0,250.0,10.0,1011.0,0.0,2.6
20080529,2016-12-31 23:00:00,fid_99942,ing_w,79.3100,12733.749695,Obery Farms,location 5489,16.7,4.0,250.0,10.0,1011.0,0.0,2.6
20080530,2016-12-31 23:00:00,fid_99942,ing_x,72.6230,12733.749695,Obery Farms,location 5489,16.7,4.0,250.0,10.0,1011.0,0.0,2.6


In [0]:
final_merge=load('/content/drive/My Drive/Phd/train_data_removied_outliers_raw.joblib')
farm_id_traget_encoding=load('/content/drive/My Drive/Phd/farm_id_target_encoding.joblib')
farming_company_weights=load('/content/drive/My Drive/Phd/farming_company_weights.joblib')
deidentified_location_weights=load('/content/drive/My Drive/Phd/deidentified_location_weights.joblib')
global_time_series_full=load('/content/drive/My Drive/Phd/global_time_series_full.joblib')

In [0]:
final_merge=pd.merge(final_merge,farm_id_traget_encoding,on=['farm_id','ingredient_type'],how='left')

In [0]:
final_merge=pd.merge(final_merge,farming_company_weights,on=['farming_company','ingredient_type'],how='left')

In [0]:
final_merge=pd.merge(final_merge,deidentified_location_weights,on=['deidentified_location','ingredient_type'],how='left')

In [0]:
final_merge['timestamp']=final_merge['timestamp'].astype('datetime64[ns]')
global_time_series_full['timestamp']=global_time_series_full['timestamp'].astype('datetime64[ns]')
final_merge=pd.merge(final_merge,global_time_series_full,on=['timestamp','ingredient_type'],how='left')

In [75]:
final_merge

,timestamp,farm_id,ingredient_type,yield,farm_area,farming_company,deidentified_location,temp_obs,cloudiness,wind_direction,dew_temp,pressure_sea_level,precipitation,wind_speed,yeild_nan_count,farm_id_ingredient_type_traget_encoding,farming_company_weights,deidentified_location_weights,globaltime_yield_ingredient_weights,globaltime_month12_ingredient_weights,globaltime_day31_ingredient_weights,globaltime_hour24_ingredient_weights,globaltime_weekday7_ingredient_weights,globaltime_quarter4_ingredient_weights,globaltime_weekofyear53_ingredient_weights
0,2016-01-01 00:00:00,fid_100053,ing_w,37.0400,2043.866000,Del Monte Foods,location 5290,10.0,8.0,350.0,2.2,1021.1,0.0,4.1,346,4.722934e+05,1.294046e+08,2.545410e+12,135023.2208,1.391212e+08,6.027066e+07,6.955550e+07,2.719701e+08,3.942612e+08,1.182668e+07
1,2016-01-01 00:00:00,fid_100107,ing_w,291.8000,35416.574563,Del Monte Foods,location 2532,-8.3,8.0,250.0,-12.2,1023.3,-1.0,4.1,0,4.773916e+06,1.294046e+08,2.228210e+12,135023.2208,1.391212e+08,6.027066e+07,6.955550e+07,2.719701e+08,3.942612e+08,1.182668e+07
2,2016-01-01 00:00:00,fid_100107,ing_x,25.3232,35416.574563,Del Monte Foods,location 2532,-8.3,8.0,250.0,-12.2,1023.3,-1.0,4.1,4,3.377998e+06,7.034721e+07,9.909914e+11,40713.7369,4.316221e+07,3.940342e+07,4.595626e+07,1.783589e+08,1.775693e+08,3.754670e+06
3,2016-01-01 00:00:00,fid_100135,ing_w,15.6400,1432.564260,Dole Food Company,location 5290,10.0,8.0,350.0,2.2,1021.1,0.0,4.1,3,2.196656e+05,1.330725e+08,2.545410e+12,135023.2208,1.391212e+08,6.027066e+07,6.955550e+07,2.719701e+08,3.942612e+08,1.182668e+07
4,2016-01-01 00:00:00,fid_100249,ing_w,19.4900,941.571905,Wayne Farms,location 5489,15.6,6.0,270.0,-5.6,1015.3,0.0,3.6,1,1.574181e+05,3.514528e+08,1.456943e+12,135023.2208,1.391212e+08,6.027066e+07,6.955550e+07,2.719701e+08,3.942612e+08,1.182668e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19478112,2016-12-31 23:00:00,fid_99921,ing_w,142.3700,15232.282977,Wayne Farms,location 5489,16.7,4.0,250.0,10.0,1011.0,0.0,2.6,3,1.285013e+06,3.514528e+08,1.456943e+12,186261.5612,1.600926e+08,3.573786e+07,7.122608e+07,2.500238e+08,4.893477e+08,2.899149e+07
19478113,2016-12-31 23:00:00,fid_99921,ing_x,102.3400,15232.282977,Wayne Farms,location 5489,16.7,4.0,250.0,10.0,1011.0,0.0,2.6,3,2.144410e+06,2.548563e+08,2.043693e+12,72540.8947,6.060628e+07,2.421878e+07,4.718487e+07,1.667514e+08,2.431905e+08,1.227002e+07
19478114,2016-12-31 23:00:00,fid_99942,ing_w,79.3100,12733.749695,Obery Farms,location 5489,16.7,4.0,250.0,10.0,1011.0,0.0,2.6,2,1.313502e+06,9.308472e+08,1.456943e+12,186261.5612,1.600926e+08,3.573786e+07,7.122608e+07,2.500238e+08,4.893477e+08,2.899149e+07
19478115,2016-12-31 23:00:00,fid_99942,ing_x,72.6230,12733.749695,Obery Farms,location 5489,16.7,4.0,250.0,10.0,1011.0,0.0,2.6,1,2.571440e+06,5.942729e+08,2.043693e+12,72540.8947,6.060628e+07,2.421878e+07,4.718487e+07,1.667514e+08,2.431905e+08,1.227002e+07


In [0]:
cat_cols=['ingredient_type','farming_company','deidentified_location']
final_merge[cat_cols] = final_merge[cat_cols].astype("category")

In [77]:

from sklearn.preprocessing import LabelEncoder
label_encoder=LabelEncoder()
le_ingredient_type=label_encoder.fit(final_merge['ingredient_type'])
final_merge['ingredient_type']=le_ingredient_type.transform(final_merge['ingredient_type'])
dump(le_ingredient_type,'/content/drive/My Drive/Phd/le_ingredient_type.joblib')

['/content/drive/My Drive/Phd/le_ingredient_type.joblib']

In [78]:

from sklearn.preprocessing import LabelEncoder
label_encoder=LabelEncoder()
le_deidentified_location=label_encoder.fit(final_merge['deidentified_location'])
final_merge['deidentified_location']=le_deidentified_location.transform(final_merge['deidentified_location'])
dump(le_deidentified_location,'/content/drive/My Drive/Phd/le_deidentified_location.joblib')

['/content/drive/My Drive/Phd/le_deidentified_location.joblib']

In [79]:

from sklearn.preprocessing import LabelEncoder
label_encoder=LabelEncoder()
le_farming_company=label_encoder.fit(final_merge['farming_company'])
final_merge['farming_company']=le_farming_company.transform(final_merge['farming_company'])
dump(le_farming_company,'/content/drive/My Drive/Phd/le_farming_company.joblib')

['/content/drive/My Drive/Phd/le_farming_company.joblib']

In [80]:
dump(final_merge,'/content/drive/My Drive/Phd/train_data_without_standarderization.joblib')

['/content/drive/My Drive/Phd/train_data_without_standarderization.joblib']

In [0]:
final_merge.drop(['timestamp','farm_id'],inplace=True,axis=1)

In [82]:
def inspect_data(data) :
      return pd.DataFrame({"Data type": data.dtypes,"NUll Values": data.isnull().sum(),
                           "No of Levels": data.apply(lambda x: str(x.nunique()),axis=0),
                          "Levels": data.apply(lambda x: str(x.unique()),axis=0)})
inspect_data(final_merge)

,Data type,NUll Values,No of Levels,Levels
ingredient_type,int64,0,4,[0. 1. 3. 2.]
yield,float64,0,1508801,[ 37.04 291.8 25.3232 ... 48.3261 578.7...
farm_area,float64,0,1385,[ 2043.866 35416.574563 1432.56426 ... 5...
farming_company,int64,0,16,[ 0. 1. 14. 11. 8. 13. 2. 6. 3. 4. 9. ...
deidentified_location,int64,0,16,[ 4. 1. 6. 8. 15. 12. 5. 13. 3. 10. 7. ...
temp_obs,float64,0,1012,[ 10. -8.3 15.6 ... -11.45 -10.95 -5.85]
cloudiness,float64,0,3909,[8. 6. 0. ... 8.259259...
wind_direction,float64,0,686,[350. 250. 270. 0. ...
dew_temp,float64,0,919,[ 2.2 -12.2 -5.6 20. ...
pressure_sea_level,float64,0,1651,[1021.1 1023.3 1015.3 ... 1043.9 1043.4 1...


Index(['timestamp', 'farm_id', 'ingredient_type', 'yield', 'farm_area',
       'farming_company', 'deidentified_location', 'temp_obs', 'cloudiness',
       'wind_direction', 'dew_temp', 'pressure_sea_level', 'precipitation',
       'wind_speed', 'yeild_nan_count',
       'farm_id_ingredient_type_traget_encoding', 'farming_company_weights',
       'deidentified_location_weights', 'globaltime_yield_ingredient_weights',
       'globaltime_month12_ingredient_weights',
       'globaltime_day31_ingredient_weights',
       'globaltime_hour24_ingredient_weights',
       'globaltime_weekday7_ingredient_weights',
       'globaltime_quarter4_ingredient_weights',
       'globaltime_weekofyear53_ingredient_weights'],
      dtype='object')

In [124]:
import time
start = time.time()
from joblib import dump,load
Training_data=load('/content/drive/My Drive/Phd/train_data_without_standarderization.joblib')

from sklearn import model_selection

X_train,X_test,Y_train,Y_test=model_selection.train_test_split(Training_data.copy().drop(['timestamp','yield','farm_id'],axis=1),Training_data['yield'],test_size=0.3,random_state=4)


from sklearn.tree import DecisionTreeRegressor
params={'max_depth': 5, 'min_samples_leaf': 100, 'min_samples_split': 4}
regressor = DecisionTreeRegressor(**params)
#regressor = DecisionTreeRegressor()
decision_tree_model=regressor.fit(X_train, Y_train)

Y_pred_train = decision_tree_model.predict(X_train)
Y_pred_test=decision_tree_model.predict(X_test)
from sklearn.metrics import mean_squared_error
mse_train=mean_squared_error(Y_train,Y_pred_train)
mse_test=mean_squared_error(Y_test,Y_pred_test)

import numpy as np 
rmse_test=np.sqrt(mse_test)
rmse_train=np.sqrt(mse_train)
print("Train RMSE")
print(np.sqrt(mse_train))
print("Test RMSE")
print(np.sqrt(mse_test))

done = time.time()
elapsed = done - start
print(elapsed)




Train RMSE
223.42144666955642
Test RMSE
223.75851760459727
137.2316653728485


In [0]:
farm_data_master=pd.read_csv('/content/drive/My Drive/Phd/new_farm_data.csv')
train_data_master=load('/content/drive/My Drive/Phd/new_test_data.joblib')
train_weather_master=pd.read_csv('/content/drive/My Drive/Phd/test_weather.csv')

,timestamp,deidentified_location,temp_obs,cloudiness,wind_direction,dew_temp,pressure_sea_level,precipitation,wind_speed
0,2017-01-01 00:00:00,location 1784,-6.1,9.0,200.0,-7.8,1017.1,0.000000,1.5
1,2017-01-01 01:00:00,location 1784,-6.1,9.0,200.0,-7.8,1017.1,0.000000,1.5
2,2017-01-01 02:00:00,location 1784,-6.7,9.0,0.0,-7.2,1017.1,0.000000,0.0
3,2017-01-01 03:00:00,location 1784,-6.7,9.0,0.0,-7.8,1016.7,0.000000,0.0
4,2017-01-01 04:00:00,location 1784,-6.1,9.0,0.0,-7.8,1016.6,0.000000,0.0
...,...,...,...,...,...,...,...,...,...
140155,2017-12-31 19:00:00,location 959,8.5,0.0,210.0,4.8,992.3,2.632517,8.2
140156,2017-12-31 20:00:00,location 959,8.5,0.0,210.0,4.5,992.1,2.632517,7.2
140157,2017-12-31 21:00:00,location 959,8.2,0.0,230.0,4.0,992.1,2.632517,10.3
140158,2017-12-31 22:00:00,location 959,7.5,0.0,260.0,4.3,993.7,2.632517,12.9


In [0]:
farm_data_master=pd.read_csv('/content/drive/My Drive/Phd/new_farm_data.csv')
test_data_master=load('/content/drive/My Drive/Phd/new_test_data.joblib')
test_weather_master=load('/content/drive/My Drive/Phd/weather_data_Testfull.joblib')

In [0]:
test_weather_master.timestamp=test_weather_master.timestamp.astype('datetime64[ns]')
test_data_master.rename(columns={'date':'timestamp'},inplace=True)
test_data_master.timestamp=test_data_master.timestamp.astype('datetime64[ns]')

In [8]:
final_merge=pd.merge(pd.merge(test_data_master, farm_data_master, on=['farm_id'], how='left'), test_weather_master, on=['timestamp', 'deidentified_location'], how='left')
dump(final_merge,'/content/drive/My Drive/Phd/test_data_raw.joblib')

['/content/drive/My Drive/Phd/test_data_raw.joblib']

In [10]:
final_merge.columns

Index(['id', 'timestamp', 'farm_id', 'ingredient_type', 'Unnamed: 0',
       'operations_commencing_year', 'num_processing_plants', 'farm_area',
       'farming_company', 'deidentified_location', 'temp_obs', 'cloudiness',
       'wind_direction', 'dew_temp', 'pressure_sea_level', 'precipitation',
       'wind_speed'],
      dtype='object')

In [0]:
final_merge.drop('Unnamed: 0',axis=1,inplace=True)

In [17]:
dump(final_merge,'/content/drive/My Drive/Phd/test_data_raw.joblib')

['/content/drive/My Drive/Phd/test_data_raw.joblib']

In [0]:
final_merge.drop(['operations_commencing_year','num_processing_plants'],axis=1,inplace=True)

In [0]:
final_merge.rename(columns={'timestamp':'timestamp2017'},inplace=True)

In [0]:
global_time_series_full=load('/content/drive/My Drive/Phd/global_time_series_full.joblib')

In [0]:
temp_timestamp_list_2017=[]
for row_number in range(global_time_series_full.shape[0]):
  temp_timestamp_list_2017.append(global_time_series_full['timestamp'].iloc[row_number][0:3]+"7"+global_time_series_full['timestamp'].iloc[row_number][4:])

In [0]:
global_time_series_full['timestamp2017']=pd.Series(temp_timestamp_list_2017)

In [24]:
global_time_series_full

,timestamp,ingredient_type,globaltime_yield_ingredient_weights,globaltime_month12_ingredient_weights,globaltime_day31_ingredient_weights,globaltime_hour24_ingredient_weights,globaltime_weekday7_ingredient_weights,globaltime_quarter4_ingredient_weights,globaltime_weekofyear53_ingredient_weights,timestamp2017
0,2016-01-01 00:00:00,ing_w,135023.2208,1.391212e+08,6.027066e+07,6.955550e+07,2.719701e+08,3.942612e+08,1.182668e+07,2017-01-01 00:00:00
1,2016-01-01 01:00:00,ing_w,154823.1326,1.391212e+08,6.027066e+07,6.858095e+07,2.719701e+08,3.942612e+08,1.182668e+07,2017-01-01 01:00:00
2,2016-01-01 02:00:00,ing_w,154960.5384,1.391212e+08,6.027066e+07,6.773713e+07,2.719701e+08,3.942612e+08,1.182668e+07,2017-01-01 02:00:00
3,2016-01-01 03:00:00,ing_w,155335.7508,1.391212e+08,6.027066e+07,6.727785e+07,2.719701e+08,3.942612e+08,1.182668e+07,2017-01-01 03:00:00
4,2016-01-01 04:00:00,ing_w,155747.4570,1.391212e+08,6.027066e+07,6.716471e+07,2.719701e+08,3.942612e+08,1.182668e+07,2017-01-01 04:00:00
...,...,...,...,...,...,...,...,...,...,...
35131,2016-12-31 19:00:00,ing_z,38584.3909,2.993657e+07,4.813706e+06,1.018745e+07,3.737245e+07,6.996550e+07,5.635746e+06,2017-12-31 19:00:00
35132,2016-12-31 20:00:00,ing_z,33645.3734,2.993657e+07,4.813706e+06,1.012459e+07,3.737245e+07,6.996550e+07,5.635746e+06,2017-12-31 20:00:00
35133,2016-12-31 21:00:00,ing_z,40044.3898,2.993657e+07,4.813706e+06,1.016599e+07,3.737245e+07,6.996550e+07,5.635746e+06,2017-12-31 21:00:00
35134,2016-12-31 22:00:00,ing_z,39119.0558,2.993657e+07,4.813706e+06,1.030854e+07,3.737245e+07,6.996550e+07,5.635746e+06,2017-12-31 22:00:00


In [0]:
farm_id_traget_encoding=load('/content/drive/My Drive/Phd/farm_id_target_encoding.joblib')
farming_company_weights=load('/content/drive/My Drive/Phd/farming_company_weights.joblib')
deidentified_location_weights=load('/content/drive/My Drive/Phd/deidentified_location_weights.joblib')

In [29]:
global_time_series_full

timestamp                                      object
ingredient_type                                object
globaltime_yield_ingredient_weights           float64
globaltime_month12_ingredient_weights         float64
globaltime_day31_ingredient_weights           float64
globaltime_hour24_ingredient_weights          float64
globaltime_weekday7_ingredient_weights        float64
globaltime_quarter4_ingredient_weights        float64
globaltime_weekofyear53_ingredient_weights    float64
timestamp2017                                  object
dtype: object

In [0]:
global_time_series_full['timestamp2017']=global_time_series_full['timestamp2017'].astype('datetime64[ns]')

In [0]:
final_merge['timestamp2017']=final_merge['timestamp2017'].astype('datetime64[ns]')

In [0]:
final_merge=pd.merge(final_merge,global_time_series_full,on=['timestamp2017','ingredient_type'])

Index(['id', 'timestamp2017', 'farm_id', 'ingredient_type', 'farm_area',
       'farming_company', 'deidentified_location', 'temp_obs', 'cloudiness',
       'wind_direction', 'dew_temp', 'pressure_sea_level', 'precipitation',
       'wind_speed', 'timestamp', 'globaltime_yield_ingredient_weights',
       'globaltime_month12_ingredient_weights',
       'globaltime_day31_ingredient_weights',
       'globaltime_hour24_ingredient_weights',
       'globaltime_weekday7_ingredient_weights',
       'globaltime_quarter4_ingredient_weights',
       'globaltime_weekofyear53_ingredient_weights'],
      dtype='object')

In [0]:
final_merge=pd.merge(final_merge,farm_id_traget_encoding,on=['farm_id','ingredient_type'],how='left')

In [106]:
final_merge.columns

Index(['id', 'timestamp2017', 'farm_id', 'ingredient_type', 'farm_area',
       'farming_company', 'deidentified_location', 'temp_obs', 'cloudiness',
       'wind_direction', 'dew_temp', 'pressure_sea_level', 'precipitation',
       'wind_speed', 'timestamp', 'globaltime_yield_ingredient_weights',
       'globaltime_month12_ingredient_weights',
       'globaltime_day31_ingredient_weights',
       'globaltime_hour24_ingredient_weights',
       'globaltime_weekday7_ingredient_weights',
       'globaltime_quarter4_ingredient_weights',
       'globaltime_weekofyear53_ingredient_weights', 'yeild_nan_count',
       'farm_id_ingredient_type_traget_encoding'],
      dtype='object')

In [0]:
final_merge=pd.merge(final_merge,farming_company_weights,on=['farming_company','ingredient_type'],how='left')

In [0]:
final_merge=pd.merge(final_merge,deidentified_location_weights,on=['deidentified_location','ingredient_type'],how='left')

In [111]:
final_merge.columns

Index(['id', 'timestamp2017', 'farm_id', 'ingredient_type', 'farm_area',
       'farming_company', 'deidentified_location', 'temp_obs', 'cloudiness',
       'wind_direction', 'dew_temp', 'pressure_sea_level', 'precipitation',
       'wind_speed', 'timestamp', 'globaltime_yield_ingredient_weights',
       'globaltime_month12_ingredient_weights',
       'globaltime_day31_ingredient_weights',
       'globaltime_hour24_ingredient_weights',
       'globaltime_weekday7_ingredient_weights',
       'globaltime_quarter4_ingredient_weights',
       'globaltime_weekofyear53_ingredient_weights', 'yeild_nan_count',
       'farm_id_ingredient_type_traget_encoding', 'farming_company_weights',
       'deidentified_location_weights'],
      dtype='object')

In [0]:
final_merge.drop(['timestamp','timestamp2017'],inplace=True,axis=1)

In [0]:
final_merge.drop(['farm_id'],inplace=True,axis=1)

In [116]:
dump(final_merge,'/content/drive/My Drive/Phd/test_data_final_model.joblib')

['/content/drive/My Drive/Phd/test_data_final_model.joblib']

In [117]:
final_merge

,id,ingredient_type,farm_area,farming_company,deidentified_location,temp_obs,cloudiness,wind_direction,dew_temp,pressure_sea_level,precipitation,wind_speed,globaltime_yield_ingredient_weights,globaltime_month12_ingredient_weights,globaltime_day31_ingredient_weights,globaltime_hour24_ingredient_weights,globaltime_weekday7_ingredient_weights,globaltime_quarter4_ingredient_weights,globaltime_weekofyear53_ingredient_weights,yeild_nan_count,farm_id_ingredient_type_traget_encoding,farming_company_weights,deidentified_location_weights
0,1,ing_w,252.696160,Obery Farms,location 7369,17.8,4.000000,100.0,11.7,1021.4,0.0,3.6,135023.2208,1.391212e+08,6.027066e+07,6.955550e+07,2.719701e+08,3.942612e+08,1.182668e+07,0,6.576176e+05,9.308472e+08,1.787181e+12
1,2,ing_w,499.446528,Obery Farms,location 7369,17.8,4.000000,100.0,11.7,1021.4,0.0,3.6,135023.2208,1.391212e+08,6.027066e+07,6.955550e+07,2.719701e+08,3.942612e+08,1.182668e+07,0,1.278194e+05,9.308472e+08,1.787181e+12
2,3,ing_w,2200.407555,Obery Farms,location 7369,17.8,4.000000,100.0,11.7,1021.4,0.0,3.6,135023.2208,1.391212e+08,6.027066e+07,6.955550e+07,2.719701e+08,3.942612e+08,1.182668e+07,0,2.069071e+06,9.308472e+08,1.787181e+12
3,4,ing_w,10833.140121,Obery Farms,location 7369,17.8,4.000000,100.0,11.7,1021.4,0.0,3.6,135023.2208,1.391212e+08,6.027066e+07,6.955550e+07,2.719701e+08,3.942612e+08,1.182668e+07,0,8.574482e+06,9.308472e+08,1.787181e+12
4,5,ing_w,743.224000,Obery Farms,location 7369,17.8,4.000000,100.0,11.7,1021.4,0.0,3.6,135023.2208,1.391212e+08,6.027066e+07,6.955550e+07,2.719701e+08,3.942612e+08,1.182668e+07,0,1.571848e+05,9.308472e+08,1.787181e+12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999994,999988,ing_x,6326.136882,Obery Farms,location 7369,21.7,4.333333,260.0,18.9,1012.6,0.0,2.1,76716.0832,8.011052e+07,4.100505e+07,4.718487e+07,1.667514e+08,2.431905e+08,1.785528e+07,1516,3.389704e+06,5.942729e+08,4.216557e+11
999995,999990,ing_x,1001.401437,Wayne Farms,location 7369,21.7,4.333333,260.0,18.9,1012.6,0.0,2.1,76716.0832,8.011052e+07,4.100505e+07,4.718487e+07,1.667514e+08,2.431905e+08,1.785528e+07,1515,1.042423e+06,2.548563e+08,4.216557e+11
999996,999994,ing_x,18667.278499,Dole Food Company,location 7369,21.7,4.333333,260.0,18.9,1012.6,0.0,2.1,76716.0832,8.011052e+07,4.100505e+07,4.718487e+07,1.667514e+08,2.431905e+08,1.785528e+07,1540,4.431793e+06,1.131742e+08,4.216557e+11
999997,999996,ing_x,18667.278499,Dole Food Company,location 7369,21.7,4.333333,260.0,18.9,1012.6,0.0,2.1,76716.0832,8.011052e+07,4.100505e+07,4.718487e+07,1.667514e+08,2.431905e+08,1.785528e+07,1851,2.613581e+06,1.131742e+08,4.216557e+11


In [126]:
X_test.columns

Index(['ingredient_type', 'farm_area', 'farming_company',
       'deidentified_location', 'temp_obs', 'cloudiness', 'wind_direction',
       'dew_temp', 'pressure_sea_level', 'precipitation', 'wind_speed',
       'yeild_nan_count', 'farm_id_ingredient_type_traget_encoding',
       'farming_company_weights', 'deidentified_location_weights',
       'globaltime_yield_ingredient_weights',
       'globaltime_month12_ingredient_weights',
       'globaltime_day31_ingredient_weights',
       'globaltime_hour24_ingredient_weights',
       'globaltime_weekday7_ingredient_weights',
       'globaltime_quarter4_ingredient_weights',
       'globaltime_weekofyear53_ingredient_weights'],
      dtype='object')

In [127]:
final_merge.columns

Index(['id', 'ingredient_type', 'farm_area', 'farming_company',
       'deidentified_location', 'temp_obs', 'cloudiness', 'wind_direction',
       'dew_temp', 'pressure_sea_level', 'precipitation', 'wind_speed',
       'globaltime_yield_ingredient_weights',
       'globaltime_month12_ingredient_weights',
       'globaltime_day31_ingredient_weights',
       'globaltime_hour24_ingredient_weights',
       'globaltime_weekday7_ingredient_weights',
       'globaltime_quarter4_ingredient_weights',
       'globaltime_weekofyear53_ingredient_weights', 'yeild_nan_count',
       'farm_id_ingredient_type_traget_encoding', 'farming_company_weights',
       'deidentified_location_weights'],
      dtype='object')

In [0]:
final_merge=final_merge.reindex(['ingredient_type', 'farm_area', 'farming_company',
       'deidentified_location', 'temp_obs', 'cloudiness', 'wind_direction',
       'dew_temp', 'pressure_sea_level', 'precipitation', 'wind_speed',
       'yeild_nan_count', 'farm_id_ingredient_type_traget_encoding',
       'farming_company_weights', 'deidentified_location_weights',
       'globaltime_yield_ingredient_weights',
       'globaltime_month12_ingredient_weights',
       'globaltime_day31_ingredient_weights',
       'globaltime_hour24_ingredient_weights',
       'globaltime_weekday7_ingredient_weights',
       'globaltime_quarter4_ingredient_weights',
       'globaltime_weekofyear53_ingredient_weights','id'],axis=1)

In [0]:
final_merge.drop('id',inplace=True,axis=1)

,ingredient_type,farm_area,farming_company,deidentified_location,temp_obs,cloudiness,wind_direction,dew_temp,pressure_sea_level,precipitation,wind_speed,yeild_nan_count,farm_id_ingredient_type_traget_encoding,farming_company_weights,deidentified_location_weights,globaltime_yield_ingredient_weights,globaltime_month12_ingredient_weights,globaltime_day31_ingredient_weights,globaltime_hour24_ingredient_weights,globaltime_weekday7_ingredient_weights,globaltime_quarter4_ingredient_weights,globaltime_weekofyear53_ingredient_weights
0,ing_w,252.696160,Obery Farms,location 7369,17.8,4.000000,100.0,11.7,1021.4,0.0,3.6,0,6.576176e+05,9.308472e+08,1.787181e+12,135023.2208,1.391212e+08,6.027066e+07,6.955550e+07,2.719701e+08,3.942612e+08,1.182668e+07
1,ing_w,499.446528,Obery Farms,location 7369,17.8,4.000000,100.0,11.7,1021.4,0.0,3.6,0,1.278194e+05,9.308472e+08,1.787181e+12,135023.2208,1.391212e+08,6.027066e+07,6.955550e+07,2.719701e+08,3.942612e+08,1.182668e+07
2,ing_w,2200.407555,Obery Farms,location 7369,17.8,4.000000,100.0,11.7,1021.4,0.0,3.6,0,2.069071e+06,9.308472e+08,1.787181e+12,135023.2208,1.391212e+08,6.027066e+07,6.955550e+07,2.719701e+08,3.942612e+08,1.182668e+07
3,ing_w,10833.140121,Obery Farms,location 7369,17.8,4.000000,100.0,11.7,1021.4,0.0,3.6,0,8.574482e+06,9.308472e+08,1.787181e+12,135023.2208,1.391212e+08,6.027066e+07,6.955550e+07,2.719701e+08,3.942612e+08,1.182668e+07
4,ing_w,743.224000,Obery Farms,location 7369,17.8,4.000000,100.0,11.7,1021.4,0.0,3.6,0,1.571848e+05,9.308472e+08,1.787181e+12,135023.2208,1.391212e+08,6.027066e+07,6.955550e+07,2.719701e+08,3.942612e+08,1.182668e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999994,ing_x,6326.136882,Obery Farms,location 7369,21.7,4.333333,260.0,18.9,1012.6,0.0,2.1,1516,3.389704e+06,5.942729e+08,4.216557e+11,76716.0832,8.011052e+07,4.100505e+07,4.718487e+07,1.667514e+08,2.431905e+08,1.785528e+07
999995,ing_x,1001.401437,Wayne Farms,location 7369,21.7,4.333333,260.0,18.9,1012.6,0.0,2.1,1515,1.042423e+06,2.548563e+08,4.216557e+11,76716.0832,8.011052e+07,4.100505e+07,4.718487e+07,1.667514e+08,2.431905e+08,1.785528e+07
999996,ing_x,18667.278499,Dole Food Company,location 7369,21.7,4.333333,260.0,18.9,1012.6,0.0,2.1,1540,4.431793e+06,1.131742e+08,4.216557e+11,76716.0832,8.011052e+07,4.100505e+07,4.718487e+07,1.667514e+08,2.431905e+08,1.785528e+07
999997,ing_x,18667.278499,Dole Food Company,location 7369,21.7,4.333333,260.0,18.9,1012.6,0.0,2.1,1851,2.613581e+06,1.131742e+08,4.216557e+11,76716.0832,8.011052e+07,4.100505e+07,4.718487e+07,1.667514e+08,2.431905e+08,1.785528e+07


In [0]:
le_deidentified_location=load('/content/drive/My Drive/Phd/le_deidentified_location.joblib')
le_farming_company=load('/content/drive/My Drive/Phd/le_farming_company.joblib')
le_ingredient_type=load('/content/drive/My Drive/Phd/le_ingredient_type.joblib')

In [0]:
final_merge['deidentified_location']=le_deidentified_location.transform(final_merge['deidentified_location'])
final_merge['farming_company']=le_farming_company.transform(final_merge['farming_company'])
final_merge['ingredient_type']=le_ingredient_type.transform(final_merge['ingredient_type'])

In [136]:
dump(final_merge,'/content/drive/My Drive/Phd/test_data_final_model.joblib')

['/content/drive/My Drive/Phd/test_data_final_model.joblib']

In [137]:
decision_tree_model.predict(final_merge)

array([ 91.43483036,   8.61124043, 218.04090095, ..., 819.33736019,
       616.11095985, 127.47258201])

In [1]:
import time
start = time.time()
from joblib import dump,load
Training_data=load('/content/drive/My Drive/Phd/train_data_without_standarderization.joblib')

from sklearn import model_selection

X_train,X_test,Y_train,Y_test=model_selection.train_test_split(Training_data.copy().drop(['timestamp','yield','farm_id'],axis=1),Training_data['yield'],test_size=0.3,random_state=4)

from sklearn.ensemble import RandomForestRegressor

print("Train validation split successful")
params={'n_estimators': 8, 'min_samples_split': 4 , 'n_jobs' : -1 ,'min_samples_leaf': 3}
regressor = RandomForestRegressor(**params)
#regressor = RandomForestRegressor(n_jobs=-1)

random_forest_tree_model=regressor.fit(X_train, Y_train)

print("Model fit complete")

Y_pred_train = random_forest_tree_model.predict(X_train)
Y_pred_test=random_forest_tree_model.predict(X_test)

from sklearn.metrics import mean_squared_error
mse_train=mean_squared_error(Y_train,Y_pred_train)
mse_test=mean_squared_error(Y_test,Y_pred_test)

import numpy as np 
rmse_test=np.sqrt(mse_test)
rmse_train=np.sqrt(mse_train)
print("Train RMSE")
print(np.sqrt(mse_train))
print("Validation RMSE")
print(np.sqrt(mse_test))

done = time.time()
elapsed = done - start
print(elapsed)



Train validation split successful
Model fit complete
Train RMSE
36.98652512669104
Validation RMSE
85.9151935525982
586.8037106990814


In [0]:
Training_data=load('/content/drive/My Drive/Phd/train_data_without_standarderization.joblib')

from sklearn.ensemble import RandomForestRegressor
params={'n_estimators': 8, 'min_samples_split': 4 , 'n_jobs' : -1 ,'min_samples_leaf': 3}
regressor = RandomForestRegressor(**params)

randomforest_model=regressor.fit(Training_data.copy().drop(['timestamp','yield','farm_id'],axis=1),Training_data['yield'])
final_merge=load('/content/drive/My Drive/Phd/test_data_final_model.joblib')
decision_tree_model_df=pd.DataFrame(columns=['id','yield'])
decision_tree_model_df['id']=pd.read_csv('/content/drive/My Drive/Phd/sample_submission1-1578562773139.csv')['id']
decision_tree_model_df['yield']=pd.Series(randomforest_model.predict(final_merge))
decision_tree_model_df.to_csv('/content/drive/My Drive/Phd/last.csv',index=False)

In [0]:
decision_tree_model_df=pd.DataFrame(columns=['id','yield'])
decision_tree_model_df['id']=pd.read_csv('/content/drive/My Drive/Phd/sample_submission1-1578562773139.csv')['id']
decision_tree_model_df['yield']=pd.Series(randomforest_model.predict(final_merge))
decision_tree_model_df.to_csv('/content/drive/My Drive/Phd/mom_and_dad.csv',index=False)

In [12]:
decision_tree_model_df

,id,yield
0,1,9.936648
1,2,16.227296
2,3,0.000000
3,4,0.000000
4,5,2.402887
...,...,...
999994,999995,308.980043
999995,999996,98.913800
999996,999997,1942.823367
999997,999998,1373.946875
